# Filter airborne Medusa data
- R Program to filter aircraft data for strong local continental influences, subtract off NOAA in situ SPO, and write out flat text files

In [19]:
library('ncdf4')
library('yaml')

In [20]:
project_tmpdir_obs = read_yaml('../_config_calc.yml')$project_tmpdir_obs
username = Sys.info()['user']
project_tmpdir_obs = gsub('\\{\\{env\\[\'USER\'\\]\\}\\}', username, project_tmpdir_obs)

In [21]:
# read in preprocessed aircraft files from read_aircraft_med.r
load('HIPPO_MED.RData')
load('ORCAS_MED.RData')
load('ATom_MED.RData')

In [22]:
# calculate datetime variables
hippodt=ISOdatetime(hippomerge$year,hippomerge$mon,hippomerge$day,hippomerge$hour,hippomerge$min,hippomerge$sec,tz='UTC')
orcasdt=ISOdatetime(orcasmerge$year,orcasmerge$mon,orcasmerge$day,orcasmerge$hour,orcasmerge$min,orcasmerge$sec,tz='UTC')
atomdt=ISOdatetime(atommerge$year,atommerge$mon,atommerge$day,atommerge$hour,atommerge$min,atommerge$sec,tz='UTC')

In [23]:
# read in NOAA in situ record from SPO
sponc=nc_open(paste(project_tmpdir_obs,'/obspack_co2_1_GLOBALVIEWplus_v6.0_2020-09-11/data/nc/co2_spo_surface-insitu_1_allvalid.nc',sep=''))
spoco2=data.frame(cbind(ncvar_get(sponc,'time_decimal'),t(ncvar_get(sponc,'time_components')),ncvar_get(sponc,'value')*1E6)) ; colnames(spoco2)=c('date','year','mon','day','hour','min','sec','co2')
qcflag=ncvar_get(sponc,'qcflag'); spoco2$co2[substr(qcflag,1,1)!='.']=NA; spoco2$co2[substr(qcflag,2,2)!='.']=NA
spodt=ISOdatetime(spoco2$year,spoco2$mon,spoco2$day,spoco2$hour,spoco2$min,spoco2$sec,tz='UTC')

# HIPPO

In [24]:
# filter
ints=read.table(paste(project_tmpdir_obs,'/hippo_xsect_filt_datetime.txt',sep=''),header=T) 
startdt=ISOdatetime(ints$startyear,ints$startmon,ints$startday,ints$starthour,ints$startmin,ints$startsec,tz='UTC')
stopdt=ISOdatetime(ints$stopyear,ints$stopmon,ints$stopday,ints$stophour,ints$stopmin,ints$stopsec,tz='UTC')
blfilt=rep(T,nrow(hippomerge))
for(i in c(1:nrow(ints))){
    blfilt[difftime(hippodt,startdt[i])>=0&difftime(hippodt,stopdt[i])<=0]=F
}
hippodt=hippodt[blfilt]
hippomerge=hippomerge[blfilt,]
print(paste('Filtered ',sum(!blfilt),' of ',length(blfilt),' HIPPO obs (',round(sum(!blfilt)/length(blfilt)*100,1),'%)',sep=''))

[1] "Filtered 12 of 1690 HIPPO obs (0.7%)"


In [25]:
# calculate differences
hippomerge$co2mspo=round(hippomerge$co2-approx(as.POSIXct(spodt),spoco2$co2,as.POSIXct(hippodt))$y,3) ## co2 = 'CO2_MED'
hippomerge$co2mqcls=round(hippomerge$co2-hippomerge$co2qcls,3)
hippomerge$co2moms=round(hippomerge$co2-hippomerge$co2oms,3)
hippomerge$co2mao2=round(hippomerge$co2-hippomerge$co2ao2,3)

In [26]:
# write out
write(names(hippomerge),'../data/aircraft-obs/HIPPO_SO_mSPO_medusa.txt',ncol=ncol(hippomerge))
write(t(hippomerge),'../data/aircraft-obs/HIPPO_SO_mSPO_medusa.txt',ncol=ncol(hippomerge),append=T)

print(apply(!is.na(hippomerge),2,sum))

    year      mon      day     hour      min      sec     camp      flt 
    1678     1677     1677     1677     1677     1677     1678     1678 
    prof      lat      lon      alt pressure    theta      co2  co2qcls 
    1677     1677     1677     1677     1677     1677     1558     1129 
  co2oms   co2ao2    strat   h2oref   n2oref    o3ref  co2mspo co2mqcls 
    1167     1432     1678     1672     1542     1676     1558     1050 
 co2moms  co2mao2 
    1090     1334 


# ORCAS

In [27]:
# filter
ints=read.table(paste(project_tmpdir_obs,'/orcas_xsect_filt_datetime.txt',sep=''),header=T)
startdt=ISOdatetime(ints$startyear,ints$startmon,ints$startday,ints$starthour,ints$startmin,ints$startsec,tz='UTC')
stopdt=ISOdatetime(ints$stopyear,ints$stopmon,ints$stopday,ints$stophour,ints$stopmin,ints$stopsec,tz='UTC')
blfilt=rep(T,nrow(orcasmerge))
for(i in c(1:nrow(ints))){
    blfilt[difftime(orcasdt,startdt[i])>=0&difftime(orcasdt,stopdt[i])<=0]=F
}
orcasdt=orcasdt[blfilt]
orcasmerge=orcasmerge[blfilt,]
print(paste('Filtered ',sum(!blfilt),' of ',length(blfilt),' ORCAS obs (',round(sum(!blfilt)/length(blfilt)*100,1),'%)',sep=''))

[1] "Filtered 9 of 392 ORCAS obs (2.3%)"


In [28]:
# calculate differences
orcasmerge$co2mspo=round(orcasmerge$co2-approx(as.POSIXct(spodt),spoco2$co2,as.POSIXct(orcasdt))$y,2) ## co2 = 'CO2_MED'
orcasmerge$co2mqcls=round(orcasmerge$co2-orcasmerge$co2qcls,3)
orcasmerge$co2mx=round(orcasmerge$co2-orcasmerge$co2x,3)
orcasmerge$co2mnoaa=round(orcasmerge$co2-orcasmerge$co2noaa,3)
orcasmerge$co2mao2=round(orcasmerge$co2-orcasmerge$co2ao2,3)

In [29]:
# write out
write(names(orcasmerge),'../data/aircraft-obs/ORCAS_SO_mSPO_medusa.txt',ncol=ncol(orcasmerge))
write(t(orcasmerge),'../data/aircraft-obs/ORCAS_SO_mSPO_medusa.txt',ncol=ncol(orcasmerge),append=T)

print(apply(!is.na(orcasmerge),2,sum))

    year      mon      day     hour      min      sec      flt     prof 
     383      383      383      383      383      383      383      383 
     lat      lon      alt pressure    theta      co2     co2x  co2qcls 
     383      383      383      383      383      340      361      162 
 co2noaa   co2ao2    strat   h2oref   n2oref  co2mspo co2mqcls    co2mx 
     361      337      383      383      366      340      138      320 
co2mnoaa  co2mao2 
     320      299 


# ATom

In [30]:
# filter
ints=read.table(paste(project_tmpdir_obs,'/atom_xsect_filt_datetime.txt',sep=''),header=T)
startdt=ISOdatetime(ints$startyear,ints$startmon,ints$startday,ints$starthour,ints$startmin,ints$startsec,tz='UTC')
stopdt=ISOdatetime(ints$stopyear,ints$stopmon,ints$stopday,ints$stophour,ints$stopmin,ints$stopsec,tz='UTC')
blfilt=rep(T,nrow(atommerge))
for(i in c(1:nrow(ints))){
    blfilt[difftime(atomdt,startdt[i])>=0&difftime(atomdt,stopdt[i])<=0]=F
}
atomdt=atomdt[blfilt]
atommerge=atommerge[blfilt,]
print(paste('Filtered ',sum(!blfilt),' of ',length(blfilt),' ATom obs (',round(sum(!blfilt)/length(blfilt)*100,1),'%)',sep=''))

[1] "Filtered 29 of 1370 ATom obs (2.1%)"


In [31]:
# calculate differences
atommerge$co2mspo=round(atommerge$co2-approx(as.POSIXct(spodt),spoco2$co2,as.POSIXct(atomdt))$y,2) ## co2 = 'CO2_MED'
atommerge$co2mqcls=round(atommerge$co2-atommerge$co2qcls,3)
atommerge$co2mao2=round(atommerge$co2-atommerge$co2ao2,3)
atommerge$co2mnoaa=round(atommerge$co2-atommerge$co2noaa,3)

In [32]:
# write out
write(names(atommerge),'../data/aircraft-obs/ATOM_SO_mSPO_medusa.txt',ncol=ncol(atommerge))
write(t(atommerge),'../data/aircraft-obs/ATOM_SO_mSPO_medusa.txt',ncol=ncol(atommerge),append=T)

print(apply(!is.na(atommerge),2,sum))

    year      mon      day     hour      min      sec     camp      flt 
    1341     1341     1341     1341     1341     1341     1341     1341 
    prof      lat      lon      alt pressure    theta      co2  co2noaa 
    1341     1341     1341     1341     1341     1341     1241     1279 
 co2qcls   co2ao2  ch4noaa  ch4qcls    strat   h2oref   n2oref    o3ref 
    1279     1314     1336      962     1341     1032      958     1337 
 co2mspo co2mqcls  co2mao2 co2mnoaa 
    1241     1188     1221     1184 
